### Question: Predicting # of Flight delays given certain weather and past flight data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in csv files from Kaggle
pressure_df = pd.read_csv('weather_data/pressure.csv')
humidity_df = pd.read_csv('weather_data/humidity.csv')
temperature_df = pd.read_csv('weather_data/temperature.csv')
wind_direction_df = pd.read_csv('weather_data/wind_direction.csv')
wind_speed_df = pd.read_csv('weather_data/wind_speed.csv')
weather_description_df = pd.read_csv("weather_data/weather_description.csv")


In [3]:
# Top 10 cities with most values in flight delay dataset
cities = ["Atlanta",'Chicago','Dallas','Charlotte','Denver','Los Angeles','Phoenix','San Francisco','Houston','New York']  
    

In [4]:
def clean_df(dataframe,name):
    dataframe = dataframe[["datetime"] + cities] # call subset of df
    dataframe_2015_index = list(dataframe.index[dataframe['datetime'] == "2015-01-01 00:00:00"])
    # going from 2015 - 2017
    dataframe = dataframe.iloc[dataframe_2015_index[0]:,:]
    # add name to columns specifying originating df. Need this specified when joining table
    new_names = dict(zip(list(cities),list(np.core.defchararray.add(cities,name))))
    new_names['datetime'] = 'datetime'
    dataframe = dataframe.rename(columns = new_names)
    return dataframe
        

In [5]:
# New cleaned and correctly labeled df
pressure_df = clean_df(pressure_df, '_pressure')
humidity_df = clean_df(humidity_df, '_humidity')
temperature_df = clean_df(temperature_df, '_temperature')
wind_direction_df = clean_df(wind_direction_df, '_wind_direction')
wind_speed_df = clean_df(wind_speed_df, '_wind_speed')
weather_description_df = clean_df(weather_description_df, '_description')

In [6]:
# Combine all dfs 
weather_df = pd.concat([pressure_df, humidity_df, temperature_df, wind_direction_df, wind_speed_df, \
           weather_description_df], axis = 1)
weather_df.head()


,datetime,Atlanta_pressure,Chicago_pressure,Dallas_pressure,Charlotte_pressure,Denver_pressure,Los Angeles_pressure,Phoenix_pressure,San Francisco_pressure,Houston_pressure,...,Atlanta_description,Chicago_description,Dallas_description,Charlotte_description,Denver_description,Los Angeles_description,Phoenix_description,San Francisco_description,Houston_description,New York_description
19716,2015-01-01 00:00:00,1042.0,1039.0,1047.0,1042.0,1046.0,1030.0,1028.0,1033.0,1043.0,...,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear,moderate rain,light rain,sky is clear,overcast clouds,sky is clear
19717,2015-01-01 01:00:00,1043.0,1037.0,1047.0,1042.0,1046.0,1030.0,1027.0,1032.0,1043.0,...,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear,moderate rain,light rain,sky is clear,overcast clouds,sky is clear
19718,2015-01-01 02:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,1030.0,1028.0,1032.0,1043.0,...,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear,moderate rain,light rain,sky is clear,overcast clouds,sky is clear
19719,2015-01-01 03:00:00,1043.0,1037.0,1046.0,1042.0,1046.0,1030.0,1028.0,1032.0,1042.0,...,sky is clear,sky is clear,broken clouds,sky is clear,sky is clear,moderate rain,light rain,sky is clear,overcast clouds,sky is clear
19720,2015-01-01 04:00:00,1043.0,1037.0,1046.0,1042.0,1047.0,1031.0,1029.0,1032.0,1042.0,...,sky is clear,sky is clear,overcast clouds,sky is clear,sky is clear,sky is clear,broken clouds,sky is clear,overcast clouds,sky is clear
